<b>Исследование данных о бронировании отелей</b> \n
<u>Шаг 1. Откройте файл с данными и изучите общую информацию</u>

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
bookings = pd.read_csv(r'C:\Users\user\Desktop\practice\2_bookings.csv',
                       encoding='windows-1251', sep=';')
bookings.info()

<class 'pandas.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Hotel                      119390 non-null  str    
 1   Is Canceled                119390 non-null  int64  
 2   Lead Time                  119390 non-null  int64  
 3   arrival full date          119390 non-null  str    
 4   Arrival Date Year          119390 non-null  int64  
 5   Arrival Date Month         119390 non-null  str    
 6   Arrival Date Week Number   119390 non-null  int64  
 7   Arrival Date Day of Month  119390 non-null  int64  
 8   Stays in Weekend nights    119390 non-null  int64  
 9   Stays in week nights       119390 non-null  int64  
 10  stays total nights         119390 non-null  int64  
 11  Adults                     119390 non-null  int64  
 12  Children                   119386 non-null  float64
 13  Babies                     119390 non-nu

<b>Выводы</b> \n
Рассмотрим полученную информацию подробнее. \n
Всего в таблице 21 столбец 119390 строк, типы данных - float64, int64, str (object) \n
Описание данных: \n
• Hotel – тип отеля (City Hotel или Resort Hotel) 
• Is canceled – бронирование было отменено (1) или нет (0); не отмененное считается успешным
• Lead time – количество дней, прошедших между датой бронирования и датой прибытия  
• Arrival full date – полная дата прибытия
• Arrival date year – год прибытия  
• Arrival date month – месяц прибытия  
• Arrival date week number – номер недели прибытия
• Arrival date day of month – день прибытия
• Stays in weekend nights – количество выходных (суббота или воскресенье), которые гость
забронировал для проживания в отеле
• Stays in week nights – количество дней (с понедельника по пятницу), которые гость забронировал
для проживания в отеле
• Stays total nights – общее число забронированных ночей (сумма двух предыдущих колонок)
• Adults – число взрослых
• Children – число детей
• Babies – число младенцев 
• Meal – выбранный тип питания
• Country – страна происхождения клиента
• Reserved room type – тип зарезервированного номера
• Assigned room type – тип полученного номера (может отличаться от забронированного)
• Customer type – тип бронирования
• Reservation status – значение последнего статуса брони: Canceled - было отменено
клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не
зарегистрировался и сообщил администрации отеля причину
• Reservation status date – дата обновления статуса \n

Количество значений в столбцах отличается. Это говорит о том, что в данных есть пропущенные значения (колонки Children, Country). \n
Для начала приведем названия столбцов к более удобному для дальнейшей работы формату, но пропуски оставим.\n

<u>Шаг 2. Предобработка данных и вывод топ-5 стран с наибольшим числом успешных бронирований</u>

In [94]:
bookings.columns = (bookings.columns.str.lower()
                                    .str.replace(' ', '_'))
bookings.loc[bookings.is_canceled == 0] \
        .country.value_counts() \
        .head()  # отбирает строки, удовлетворяющие условию "бронь не отменена", и считает количество таких броней для всех стран, выводя топ-5

country
PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: count, dtype: int64

<b>Выводы</b> \n
Названия столбцов приведены к единому формату. \n
Португалия (PRT) лидирует по количеству успешных бронирований — 21 071 запись. Это в ~2,2 раза больше, чем у страны на втором месте (Великобритания, 9 676 записей). \n

<u>Шаг 3. Определение среднего количества ночей, на которое бронируют отели разных типов</u>

In [95]:
bookings.groupby(bookings.hotel, as_index=False) \
        .stays_total_nights \
        .mean() \
        .round(2) # группировка по типу отеля, поиск среднего количества забронированных ночей и округление полученных значений

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


<b>Выводы</b> \n
Гости Resort Hotel в среднем снимают номера на ~45% ночей дольше, чем гости City Hotel, что может говорить о разных целевых аудиториях данных типов отелей (Resort Hotel - курортный отель). \n

<u>Шаг 4. Определение количества кейсов, когда тип номера, присвоенный клиенту, отличается от изначально забронированного</u>

In [96]:
bookings.loc[bookings.assigned_room_type != bookings.reserved_room_type] \
        .value_counts() 

hotel         is_canceled  lead_time  arrival_full_date  arrival_date_year  arrival_date_month  arrival_date_week_number  arrival_date_day_of_month  stays_in_weekend_nights  stays_in_week_nights  stays_total_nights  adults  children  babies  meal       country  reserved_room_type  assigned_room_type  customer_type    reservation_status  reservation_status_date
City Hotel    0            164        2015-10-02         2015               October             40                        2                          0                        2                     2                   1       0.0       0       BB         PRT      A                   D                   Transient-Party  Check-Out           2015-10-04                 35
                           69         2015-10-25         2015               October             44                        25                         2                        1                     3                   2       0.0       0       BB         PRT      A     

<b>Выводы</b> \n
Подобные наблюдения, когда номер отлиличался от первоначального, имеются, но их относительно не много - около 10% (12 116 записей). \n

<u>Шаг 5. Месяц, на который чаще всего оформляли бронь в 2016 и 2017 гг</u>

In [97]:
bookings.loc[bookings.arrival_date_year.isin([2016, 2017]), ['arrival_date_year', 'arrival_date_month']] \
         .value_counts() \
         .head(2)

arrival_date_year  arrival_date_month
2017               May                   6313
2016               October               6203
Name: count, dtype: int64

<b>Выводы</b> \n
В 2016 году гости чаще предпочитали заселяться осенью, а в следующем году большинство посетителей выбирали весну. \n

<u>Шаг 6. Месяц, на который чаще всего отменялись бронирования отеля типа City Hotel в 2015-2017 гг</u>

In [98]:
result = (bookings.loc[bookings.hotel == 'City Hotel'] 
         .groupby([bookings.arrival_date_year, bookings.arrival_date_month], as_index = False)
         .agg({'is_canceled' : 'sum'}))
result = result.loc[result.groupby('arrival_date_year')['is_canceled']
                          .idxmax()] # индекс максимального числа по столбцу счетчика отмен
print(result)

    arrival_date_year arrival_date_month  is_canceled
5                2015          September         1543
16               2016            October         1947
25               2017                May         2217


<u>Шаг 7. Каких гостей в среднем больше: взрослых, детей или младенцев? </u>

In [99]:
if bookings.adults.mean() >= bookings.children.mean() and bookings.adults.mean() >= bookings.babies.mean():
    print('Взрослые:', bookings.adults.mean().round(2))
elif bookings.children.mean() >= bookings.adults.mean() and bookings.children.mean() >= bookings.babies.mean():
    print('Дети:', bookings.children.mean().round(2))
else:
    print('Младенцы:', bookings.children.babies().round(2))

Взрослые: 1.86


<u>Шаг 8. Кто чаще заселяется в отели разных типов: дети или младенцы? </u>

In [100]:
bookings['total_kids'] = bookings[['children', 'babies']].sum(axis=1) # запись суммы кол-ва детей и младенцев в доп.столбец
bookings.groupby(['hotel']) \
        .total_kids.mean() \
        .round(2)

hotel
City Hotel      0.10
Resort Hotel    0.14
Name: total_kids, dtype: float64

<u>Шаг 9. Расчет коэффициента оттока в зависимости от наличия детей у гостей (CR) </u>

In [102]:
bookings['has_kids'] = np.where(bookings['total_kids'] == 0, False, True)
without_children = bookings.loc[(bookings.has_kids == False) & (bookings.is_canceled == 1)].is_canceled.count()
with_children = bookings.loc[(bookings.has_kids == True) & (bookings.is_canceled == 1)].is_canceled.count()
churn_woch = (100 * without_children)/(bookings.has_kids.count())
churn_wch = (100 * with_children)/(bookings.has_kids.count())
print('% оттока гостей без детей:', churn_woch.round(2))
print('% оттока гостей с детьми:', churn_wch.round(2))

% оттока гостей без детей: 34.31
% оттока гостей с детьми: 2.73


<b>Выводы</b> \n
Гости с детьми значительно реже отменяли бронирование, что может говорить о том, что семьи - более лояльная аудитория. Одним из вариантов стимулирования данного сегмента бронировать номера чаще может быть запуск таргетированных акций (скидки на длительное проживание, бесплатное питание для детей и т.д.)